# modular analysis of DeepLabCut pose estimations
## test_extract_params_from_dlc.ipynb

Analyze .h5 files output by DeepLabCut and Anipose and extract properties of mice (and prey if present). Outputs of this pipeline are saved out as .nc files and can be opened as xarrays. Run this in the ```fmephys``` environment.

Last modified September 07, 2020

**NOTE: functions have been updated to take in user inputs through a .josn config file read in as a dictionary. This jupyter notebook and the functions it directly uses in `nb_wrapper.py` aren't updated, so the basic functions in this repository won't work with this notebook. Use terminal instead**

## global user inputs

### file paths

In [ ]:
# source for .avi videos, DLC .h5 files, and .csv timestamp files
global_data_path = 'data/path/'
# path into which outputs will be saved (directory will be created if it does not already exist)
global_save_path = '/save/path/'


### trial info

In [ ]:
# number of labeled topdown DeepLabCut points (including points on cricket, if there is one)
topdown_pt_num = 10
# if there is a cricket in the tank as the last two DeepLabCut points (True/False)
cricket = True
# number of labeled eye camera points (including tear duct and back of eye, if those exist)
eye_pt_num = 8
# if there are eye points labeled for the tear duct and back of the eye (True/False)
tear = True
# use BonsaiTS timestamps (True) or FlirTS timestamps (False)
bonsaitime = True
# name of trial to run on e.g. '070920_mouse01_test01' -- do not include the file extension in the name string, the extensions '.avi', '_BonsaiTS.csv', and '.h5' will be added as needed
trial_name = 'trial_name'
# number of topdown cameras to read in; currently handles up to 3
toptrial_count = 1
# some parameters to find the pupil rotation
ranger = 10
world_interp_method = 'linear'


### thresholding and saftey

In [ ]:
# DLC likelihood threshold (default=0.99)
lik_thresh = 0.99
# max number of pixels for radius of pupil (default=50)
pxl_thresh = 50
# maximum ratio of ellipse shortaxis to longaxis (default=0.90)
ell_thresh = 0.90
# value with which to correct y-coordinates in topdown view (default=0)
coord_cor = 0


### module imports

In [ ]:
from nb_wrapper import topdown_intake, eye_intake
from util.save_data import savetrial
from util.track_world import adjust_world, find_pupil_rotation


## topdown tracking

In [ ]:
# run topdown_intake for up to three individual top-down cameras
for num in range(1,toptrial_count+1):
    trialext = 'TOP' + str(num)
    top_trialname = trial_name + '_' + trialext
    xtop = topdown_intake(global_data_path, top_trialname, trialext, global_save_path, lik_thresh, coord_cor, topdown_pt_num, cricket, bonsaitime)
    savetrial(xtop, global_save_path, trial_name, trialext)


## eye tracking and worldcam adjustment

### left eye

In [ ]:
%%time
LEye = trial_name + '_LEYE'

# find ellipse parameters
xLEye = eye_intake(global_data_path, LEye, 'LEYE', global_save_path, lik_thresh, pxl_thresh, ell_thresh, eye_pt_num, tear, bonsaitime)
savetrial(xLEye, global_save_path, trial_name, 'LEYE')


### right eye

In [ ]:
%%time
REye = trial_name + '_REYE'

# find ellipse parameters
xREye, rfit, shift = eye_intake(global_data_path, REye, 'REYE', global_save_path, lik_thresh, pxl_thresh, ell_thresh, eye_pt_num, tear, bonsaitime)
savetrial(xREye, global_save_path, trial_name, 'REYE')
